In [1]:
!nvidia-smi

Tue Sep  6 10:17:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   66C    P0    49W /  N/A |      0MiB /  6144MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Imports
import torch
from tqdm import tqdm
from torchvision import datasets, transforms

In [3]:
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.deterministic = True
# The flag below controls whether to allow TF32 on matmul. This flag defaults to False
# in PyTorch 1.12 and later.
torch.backends.cuda.matmul.allow_tf32 = True

# The flag below controls whether to allow TF32 on cuDNN. This flag defaults to True.
torch.backends.cudnn.allow_tf32 = True

# I set tf32 to be false to stop usign tensor cores, just to see if using half pres then is faster or not

In [4]:
# Download resnet 100 and put in gpu
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet101', pretrained=False)
# Replace last layer with 10 nodes
model.fc = torch.nn.Linear(2048, 10)

# Put model in gpu
model = model.cuda()

Using cache found in /home/venom/.cache/torch/hub/pytorch_vision_v0.6.0
/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/venom/miniconda3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
torch.__version__

'1.12.1+cu116'

In [6]:
# Download cifar 10 dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

trainset = datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='./data', train=False,
                           download=True, transform=transform)


Files already downloaded and verified
Files already downloaded and verified


In [7]:
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=64, shuffle=True, num_workers=4,pin_memory=True)

testloader = torch.utils.data.DataLoader(
    testset, batch_size=64, shuffle=False, num_workers=4,pin_memory=True)


In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()


In [9]:
# Acc before training

model.eval()
correct = 0
total = 0
with torch.no_grad():
    with tqdm(testloader, unit="batch") as t2epoch:
        for data, target in t2epoch:
            t2epoch.set_description("Test")
            data, target = data.cuda(), target.cuda()
            output = model(data)
            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()
            t2epoch.set_postfix(Accuracy=(100 * correct / total))


Test: 100%|██████████| 157/157 [00:05<00:00, 29.99batch/s, Accuracy=10]  


In [10]:
# Train
for epoch in range(1,11):
    model.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data, target in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                output = model(data)
                loss = criterion(output, target)
            loss.backward()
            optimizer.step()
            tepoch.set_postfix(loss=loss.item())

    # Test
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        with tqdm(testloader, unit="batch") as t2epoch:
            for data, target in t2epoch:
                t2epoch.set_description(f"Epoch {epoch}")
                data, target = data.cuda(), target.cuda()
                output = model(data)
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
                t2epoch.set_postfix(Accuracy=(100 * correct / total))

    print(" ")


Epoch 1: 100%|██████████| 157/157 [00:03<00:00, 44.36batch/s, Accuracy=25.4]


Epoch 2: 100%|██████████| 157/157 [00:03<00:00, 44.36batch/s, Accuracy=34.4]


Epoch 3: 100%|██████████| 157/157 [00:03<00:00, 44.40batch/s, Accuracy=38.4]


Epoch 4: 100%|██████████| 157/157 [00:03<00:00, 44.39batch/s, Accuracy=43.4]


Epoch 5: 100%|██████████| 157/157 [00:03<00:00, 44.53batch/s, Accuracy=45.4]


Epoch 6: 100%|██████████| 157/157 [00:03<00:00, 44.36batch/s, Accuracy=48.7]


Epoch 7: 100%|██████████| 157/157 [00:03<00:00, 44.25batch/s, Accuracy=51.7]


Epoch 8: 100%|██████████| 157/157 [00:03<00:00, 44.22batch/s, Accuracy=52]  


Epoch 9: 100%|██████████| 157/157 [00:03<00:00, 44.11batch/s, Accuracy=54.4]


Epoch 10: 100%|██████████| 157/157 [00:03<00:00, 44.15batch/s, Accuracy=55.3]

In [11]:
!nvidia-smi

Tue Sep  6 10:25:45 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.65.01    Driver Version: 515.65.01    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:01:00.0 Off |                  N/A |
| N/A   70C    P0    74W /  N/A |   2485MiB /  6144MiB |     54%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [12]:
#redefine model
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet101', pretrained=False)
model.fc = torch.nn.Linear(2048, 10)
model = model.cuda()

Using cache found in /home/venom/.cache/torch/hub/pytorch_vision_v0.6.0


In [13]:
scaler = torch.cuda.amp.GradScaler()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
criterion = torch.nn.CrossEntropyLoss()

In [14]:
# Train
for epoch in range(1,11):
    model.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data, target in tepoch:
            tepoch.set_description(f"Epoch {epoch}")
            data, target = data.cuda(), target.cuda()
            optimizer.zero_grad()
            with torch.cuda.amp.autocast():
                output = model(data)
                loss = criterion(output, target)
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            tepoch.set_postfix(loss=loss.item())

    # Test
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        with tqdm(testloader, unit="batch") as t2epoch:
            for data, target in t2epoch:
                t2epoch.set_description(f"Epoch {epoch}")
                data, target = data.cuda(), target.cuda()
                output = model(data)
                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()
                t2epoch.set_postfix(Accuracy=(100 * correct / total))

    print(" ")


Epoch 1: 100%|██████████| 157/157 [00:03<00:00, 44.80batch/s, Accuracy=25.4]


Epoch 2: 100%|██████████| 157/157 [00:03<00:00, 44.41batch/s, Accuracy=35.7]


Epoch 3: 100%|██████████| 157/157 [00:03<00:00, 44.43batch/s, Accuracy=38.9]


Epoch 4: 100%|██████████| 157/157 [00:03<00:00, 44.46batch/s, Accuracy=44.3]


Epoch 5: 100%|██████████| 157/157 [00:03<00:00, 44.74batch/s, Accuracy=46.4]


Epoch 6: 100%|██████████| 157/157 [00:03<00:00, 44.26batch/s, Accuracy=49.4]


Epoch 7: 100%|██████████| 157/157 [00:03<00:00, 44.31batch/s, Accuracy=51.7]


Epoch 8: 100%|██████████| 157/157 [00:03<00:00, 44.10batch/s, Accuracy=52.6]


Epoch 9: 100%|██████████| 157/157 [00:03<00:00, 43.96batch/s, Accuracy=54.7]


Epoch 10: 100%|██████████| 157/157 [00:03<00:00, 44.25batch/s, Accuracy=56]  